In [128]:
from pynq import Overlay
from pynq import MMIO
from pynq import allocate
import numpy as np
import math

# Program bitstream to FPGA
overlay = Overlay('./DES_axi_wrapper.bit')

# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

In [195]:
SIZE_OF_ALGO = 8 # 8 bytes = 64 bits
MAX_PARALLEL_SEND = 8

input_type = np.dtype((">V", SIZE_OF_ALGO * 2))
output_type = np.dtype((">V", SIZE_OF_ALGO))
print(f"Using {MAX_PARALLEL_SEND}x type with input: {input_type.itemsize * 8} bits, and output: {output_type.itemsize * 8} bits")

# Allocate physical memory
input_buffer = allocate(shape=(MAX_PARALLEL_SEND,), dtype=input_type)
output_buffer = allocate(shape=(MAX_PARALLEL_SEND,), dtype=output_type)

Using 8x type with input: 128 bits, and output: 64 bits


In [196]:
message = "Hello World!"
key = "BBBBBBBB"


key_bytes = bytes(key, "ascii")
assert(len(key_bytes) == 8)
print(key_bytes.hex())

message_bytes = bytes(message, "ascii")
print(message_bytes.hex())

iterations = math.ceil(len(message_bytes) / SIZE_OF_ALGO)
assert(iterations < MAX_PARALLEL_SEND)

4242424242424242
48656c6c6f20576f726c6421


In [197]:
# Write input
for i in range(iterations):
    part = message_bytes[8 * i : 8 * (i + 1)]
    input_buffer[i] = (part + bytes(SIZE_OF_ALGO - len(part)))[::-1] + key_bytes[0:8][::-1]
    
    print(f"0b{input_buffer[i]}")

0bb'\x6F\x57\x20\x6F\x6C\x6C\x65\x48\x42\x42\x42\x42\x42\x42\x42\x42'
0bb'\x00\x00\x00\x00\x21\x64\x6C\x72\x42\x42\x42\x42\x42\x42\x42\x42'


In [113]:
# # Check the written data
# for i in range(8):
#     print("Data: 0x%08X, Key: 0x%08X" % (input_buffer[i][0], input_buffer[i][1]))

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [198]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [199]:
# Check the memory content after DMA transfer
for i in range(iterations):
    print(bytes(output_buffer[i]).hex()[::-1])

bd2d6126e67a0567
1d7f2fab848b7d4b


In [10]:

# Delete buffer to prevent memory leak
del input_buffer, output_buffer